In [1]:
import torch
import torch.nn.functional as F
from torch_geometric.datasets import Planetoid
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data
import optuna
import torch_geometric.utils
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
"""
    In this project, I am building a GCN model that performs both node-level classification and regression on the Cora
    citation dataset. It includes data preprocessing like randomly removing nodes and features, model definition,
    training and evaluation with hyperparameter tuning.
    
    The hyperparameters tuned are:
    -'hidden_channels'
    -'learning_rate'
    -'weight_decay'
    * Number of epochs can also be tuned, with Cross-validation.
    
    Test Accuracy value: 0.8291 
    Test MSE value: 0.0718
    -The test accuracy value of 0.80 is good. The model can learn to classify nodes into their respective classes.
    -MSE score is relatively good. For the regression task, a lower MSE score would be desired.
    
    *** I hope to make additions and changes to this notebook later on to use it on my blog -linksomeneurons.com- while
    writing about implementation of neural networks or specifically GNN-GCN-GAN.
    
    In case of errors or improvements, do not hesitate to contact me via my website.
"""

"\n    In this project, I am building a GCN model that performs both node-level classification and regression on the Cora\n    citation dataset. It includes data preprocessing like randomly removing nodes and features, model definition,\n    training and evaluation with hyperparameter tuning.\n    \n    The hyperparameters tuned are:\n    -'hidden_channels'\n    -'learning_rate'\n    -'weight_decay'\n    * Number of epochs can also be tuned, with Cross-validation.\n    \n    *** I hope to make additions and changes to this notebook later on to use it on my blog -linksomeneurons.com- while\n    writing about implementation of neural networks or specifically GNN-GCN-GAN.\n"

In [3]:
#Loading the Cora dataset
dataset = Planetoid(root='data/Cora', name='Cora')
data = dataset[0]

In [4]:
#Randomly removing some data points to perform node-level regression later, keeping the 80%
num_nodes = data.num_nodes
keep_mask = torch.rand(num_nodes) < 0.8
data.x = data.x[keep_mask]
data.y = data.y[keep_mask]
data.edge_index, _ = torch_geometric.utils.subgraph(keep_mask, data.edge_index, None, relabel_nodes=True)

In [5]:
#Selecting a subset of features (e.g., keep 50% of the features)
num_features = data.num_features
keep_features = np.random.choice(num_features, size=num_features//2, replace=False)
data.x = data.x[:, keep_features]

In [6]:
#Assigning a synthetic continuous target for node-level prediction
continuous_target = torch.rand(data.num_nodes)

#Combining classification and regression targets
data.y = torch.stack([data.y.float(), continuous_target], dim=1)

#Data split
num_nodes = data.num_nodes
train_mask, test_mask = train_test_split(np.arange(num_nodes), test_size=0.2, random_state=42)
train_mask, val_mask = train_test_split(train_mask, test_size=0.25, random_state=42)

data.train_mask = torch.zeros(num_nodes, dtype=torch.bool)
data.train_mask[train_mask] = True
data.val_mask = torch.zeros(num_nodes, dtype=torch.bool)
data.val_mask[val_mask] = True
data.test_mask = torch.zeros(num_nodes, dtype=torch.bool)
data.test_mask[test_mask] = True

In [7]:
#The GCN model for both classification and regression
class GCN(torch.nn.Module):
    def __init__(self, num_features, num_classes, hidden_channels):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(num_features, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.classifier = torch.nn.Linear(hidden_channels, num_classes)
        self.regressor = torch.nn.Linear(hidden_channels, 1)

    def forward(self, x, edge_index):
        x = F.relu(self.conv1(x, edge_index))
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        class_output = F.log_softmax(self.classifier(x), dim=1)
        reg_output = self.regressor(x).squeeze(1)
        return class_output, reg_output

In [8]:
def train(model, optimizer, data):
    model.train()
    optimizer.zero_grad()
    class_out, reg_out = model(data.x, data.edge_index)
    class_loss = F.nll_loss(class_out[data.train_mask], data.y[data.train_mask, 0].long())
    reg_loss = F.mse_loss(reg_out[data.train_mask], data.y[data.train_mask, 1])
    loss = class_loss + reg_loss
    loss.backward()
    optimizer.step()
    return loss.item()

def evaluate(model, data, mask):
    model.eval()
    with torch.no_grad():
        class_out, reg_out = model(data.x, data.edge_index)
        class_pred = class_out[mask].argmax(dim=1)
        class_correct = (class_pred == data.y[mask, 0].long()).sum()
        class_acc = int(class_correct) / int(mask.sum())
        reg_mse = F.mse_loss(reg_out[mask], data.y[mask, 1])
    return class_acc, reg_mse.item()

In [9]:
def objective(trial):
    #Hyperparameters to optimize
    hidden_channels = trial.suggest_int('hidden_channels', 8, 64)
    learning_rate = trial.suggest_float('learning_rate', 1e-3, 1e-1, log=True)
    weight_decay = trial.suggest_float('weight_decay', 1e-5, 1e-2, log=True)

    model = GCN(data.num_features, dataset.num_classes, hidden_channels)
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

    best_val_acc = 0
    for epoch in range(200):
        loss = train(model, optimizer, data)
        val_acc, val_mse = evaluate(model, data, data.val_mask)
        
        trial.report(val_acc, epoch)
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()
        
        if val_acc > best_val_acc:
            best_val_acc = val_acc

    return best_val_acc

In [10]:
#Hyperparameter optimization
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)

print('Best trial:')
trial = study.best_trial
print(f'  Value: {trial.value}')
print('  Params: ')
for key, value in trial.params.items():
    print(f'    {key}: {value}')

#The final model with the best hyperparameters
best_params = study.best_params
final_model = GCN(data.num_features, dataset.num_classes, best_params['hidden_channels'])
final_optimizer = torch.optim.Adam(final_model.parameters(), lr=best_params['learning_rate'], weight_decay=best_params['weight_decay'])

for epoch in range(200):
    loss = train(final_model, final_optimizer, data)
    if (epoch + 1) % 10 == 0:
        train_acc, train_mse = evaluate(final_model, data, data.train_mask)
        val_acc, val_mse = evaluate(final_model, data, data.val_mask)
        print(f'Epoch: {epoch+1:03d}, Loss: {loss:.4f}, Train Acc: {train_acc:.4f}, '
              f'Train MSE: {train_mse:.4f}, Val Acc: {val_acc:.4f}, Val MSE: {val_mse:.4f}')

[I 2024-08-29 14:20:55,912] A new study created in memory with name: no-name-39cf2c3b-2e39-4f1b-9bb8-40aac8846d84
[I 2024-08-29 14:20:58,830] Trial 0 finished with value: 0.8452655889145496 and parameters: {'hidden_channels': 33, 'learning_rate': 0.0019083604310603539, 'weight_decay': 4.353289752957122e-05}. Best is trial 0 with value: 0.8452655889145496.
[I 2024-08-29 14:21:01,383] Trial 1 finished with value: 0.8475750577367206 and parameters: {'hidden_channels': 24, 'learning_rate': 0.0017478779406384542, 'weight_decay': 0.00037752261953927766}. Best is trial 1 with value: 0.8475750577367206.
[I 2024-08-29 14:21:04,391] Trial 2 finished with value: 0.8452655889145496 and parameters: {'hidden_channels': 31, 'learning_rate': 0.01741645048055879, 'weight_decay': 0.0015319354147326379}. Best is trial 1 with value: 0.8475750577367206.
[I 2024-08-29 14:21:06,740] Trial 3 finished with value: 0.8383371824480369 and parameters: {'hidden_channels': 19, 'learning_rate': 0.04102617098461099, '

Best trial:
  Value: 0.859122401847575
  Params: 
    hidden_channels: 17
    learning_rate: 0.05917446282936153
    weight_decay: 0.004473700532865897
Epoch: 010, Loss: 1.9140, Train Acc: 0.2963, Train MSE: 0.0922, Val Acc: 0.3303, Val MSE: 0.0862
Epoch: 020, Loss: 1.9273, Train Acc: 0.2963, Train MSE: 0.0908, Val Acc: 0.3303, Val MSE: 0.0879
Epoch: 030, Loss: 1.7397, Train Acc: 0.2963, Train MSE: 0.0971, Val Acc: 0.3303, Val MSE: 0.0890
Epoch: 040, Loss: 1.3591, Train Acc: 0.5170, Train MSE: 0.0904, Val Acc: 0.5150, Val MSE: 0.0889
Epoch: 050, Loss: 0.9995, Train Acc: 0.8071, Train MSE: 0.0882, Val Acc: 0.7182, Val MSE: 0.0864
Epoch: 060, Loss: 0.6421, Train Acc: 0.8858, Train MSE: 0.0837, Val Acc: 0.7714, Val MSE: 0.0781
Epoch: 070, Loss: 0.5713, Train Acc: 0.8981, Train MSE: 0.0846, Val Acc: 0.7852, Val MSE: 0.0797
Epoch: 080, Loss: 0.5394, Train Acc: 0.9213, Train MSE: 0.0832, Val Acc: 0.8222, Val MSE: 0.0793
Epoch: 090, Loss: 0.5178, Train Acc: 0.9244, Train MSE: 0.0837, Val Acc:

In [11]:
# Final evaluation
test_acc, test_mse = evaluate(final_model, data, data.test_mask)
print(f'Test Accuracy: {test_acc:.4f}, Test MSE: {test_mse:.4f}')

Test Accuracy: 0.8291, Test MSE: 0.0718
